# Práctica 3.5
### [Introducción a los sistemas inteligentes](https://fagonzalezo.github.io/)
___________

**Instrucciones de envío:**

Este notebook debe enviarse a través del siguiente [File Request](https://www.dropbox.com/request/0zhMgxCML3Q8tSG35kbE) antes del final de la clase. El archivo debe nombrarse como  `isi-practica3.5-unalusername1-unalusername2-unalusername3.ipynb`, donde unalusername es el nombre de usuario asignado por la universidad de cada uno de los miembros del grupo.

**Grupo de trabajo:**

* Juan Sebastian Muñoz Lemus jumunozle
* Julian Esteban Cadena Rojas jcadenar
* John Alejandro Pastor Sandoval jpastor

___________

## Triple Cross Puzzle

El *Triple Cross* es un rompecabezas diseñado por la empresa Binary Arts.

<img src="https://www.cs.brandeis.edu/~storer/JimPuzzles/SLIDE/PortToPort/TripleCrossPhoto-TN.jpg"
alt="Triple Cross " width="240" height="180" border="10" />

Aquí puede encontrar más detalles: https://www.jaapsch.net/puzzles/port.htm.

Su objetivo es modelar el *Triple Cross* como un problema de búsqueda y resolverlo usando diferentes algoritmos de búsqueda.

_________


### 1. Cree una clase para modelar el problema del Triple Cross

Debe utilizar la plantilla de clase que se encuentra en la siguiente celda.

#### Clase TripleCross_problem

In [ ]:
# This is the Problem class from AIMA, you don't have to modify it

class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)

    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)


class TripleCross_problem(Problem):  # Clase que modela el rompecabezas Triple Cross como problema de búsqueda

    def __init__(self, initial):  # Constructor: recibe un estado inicial
        '''
        Minimal working model of the Triple Cross puzzle.
        State: (plunger_left, plunger_right, tiles)
        - plunger_left/right: 0 (up), 1 (down)
        - tiles: tuple of 18 chars at positions Q,R,A..P as per diagram
        Actions: 'R' (shift middle area right), 'L' (shift middle area left),
                 '/' (toggle plungers and rotate plunger columns)
        '''
        self.expanded = 0  # Contador opcional de nodos/llamadas (no usado por los algoritmos)

        # Positions mapping for readability
        self.positions = {  # Mapa letra->índice para ubicar fichas en la tupla
            'Q': 0, 'R': 1,  # fila superior (plungers)
            'A': 2, 'B': 3, 'C': 4, 'D': 5, 'E': 6, 'F': 7, 'G': 8,  # fila A..G
            'H': 9, 'I': 10, 'J': 11, 'K': 12, 'L': 13, 'M': 14, 'N': 15,  # fila H..N
            'O': 16, 'P': 17  # fila inferior O,P (debajo del centro)
        }

        # Construct canonical goal per description:
        # - Left plunger down (1), right plunger up (0)
        # - Circle 2x2 at F,G,M,N
        # - Three lines at A,B,C (any order treated as identical symbol 'L')
        # - Logos at O='B' (Binary) and P='A' (Arts)
        tiles = ['E'] * 18  # Inicializamos todas las casillas como vacías 'E'
        tiles[self.positions['F']] = 'C'  # Colocamos círculos en F
        tiles[self.positions['G']] = 'C'  # y G
        tiles[self.positions['M']] = 'C'  # y M
        tiles[self.positions['N']] = 'C'  # y N (forma el cuadrado 2x2)
        tiles[self.positions['A']] = 'L'  # Colocamos líneas en A
        tiles[self.positions['B']] = 'L'  # y B
        tiles[self.positions['C']] = 'L'  # y C (orden indistinto entre sí)
        tiles[self.positions['O']] = 'B'  # Logo Binary en O
        tiles[self.positions['P']] = 'A'  # Logo Arts en P
        self.goal = (1, 0, tuple(tiles))  # Estado meta: plungers (1,0) y fichas como arriba

        # Initialize superclass
        super().__init__(initial=tuple(initial), goal=self.goal)  # Guardamos estado inicial y meta en la superclase

    def actions(self, state):  # Devuelve la lista de acciones permitidas en cualquier estado
        """All moves are always available: right, left, plungers toggle."""
        return ['R', 'L', '/']  # R: derecha, L: izquierda, /: alternar plungers y rotar columnas

    def result(self, state, action):  # Transición: aplica una acción y retorna el nuevo estado
        """
        Return the state that results from executing the given
        action at the given state. The action must be one of
        self.actions(state).
        """
        pl_left, pl_right, tiles = state  # Desempaquetamos el estado (plungers y fichas)
        new_tiles = list(tiles)  # Copiamos las fichas para modificarlas de forma inmutable

        if action == 'R':  # Acción R: mover banda central (A..G, H..N) una posición a la derecha
            # Rotate the 7x2 main area (A..G and H..N) one step to the right.
            # Esto coincide con la notación del rompecabezas: R desplaza el
            # bloque central horizontalmente (con wrap-around) hacia la derecha.
            main = new_tiles[2:16]  # Extraemos las 14 posiciones A..N
            shifted = main[-1:] + main[:-1]  # Rotación circular a la derecha
            new_tiles[2:16] = shifted  # Reinsertamos el segmento rotado
            return (pl_left, pl_right, tuple(new_tiles))  # Devolvemos nuevo estado

        if action == 'L':  # Acción L: mover banda central una posición a la izquierda
            # Rotate the 7x2 main area one step to the left (wrap-around).
            main = new_tiles[2:16]  # Extraemos A..N
            shifted = main[1:] + main[:1]  # Rotación circular a la izquierda
            new_tiles[2:16] = shifted  # Reinsertamos
            return (pl_left, pl_right, tuple(new_tiles))  # Nuevo estado

        if action == '/':  # Acción '/': alternar plungers y (si quedan abajo) rotar sus columnas
            # Toggle plungers: ambos cambian (up<->down) como en la notación '/'.
            pl_left = 1 - pl_left  # Toggle del plunger izquierdo
            pl_right = 1 - pl_right  # Toggle del plunger derecho

            # When a plunger is down, rotate its column down by one step.
            # Columnas afectadas (según diagrama de letras):
            #  - Izquierda: Q -> A -> H -> O -> Q
            #  - Derecha:  R -> G -> N -> P -> R
            if pl_left == 1:  # Si el plunger izquierdo queda ABAJO, rotamos su columna (Q,A,H,O)
                q, a, h, o = self.positions['Q'], self.positions['A'], self.positions['H'], self.positions['O']
                new_tiles[q], new_tiles[a], new_tiles[h], new_tiles[o] = (
                    new_tiles[o], new_tiles[q], new_tiles[a], new_tiles[h]  # Desplazamiento hacia abajo
                )
            if pl_right == 1:  # Si el plunger derecho queda ABAJO, rotamos su columna (R,G,N,P)
                r, g, n, p = self.positions['R'], self.positions['G'], self.positions['N'], self.positions['P']
                new_tiles[r], new_tiles[g], new_tiles[n], new_tiles[p] = (
                    new_tiles[p], new_tiles[r], new_tiles[g], new_tiles[n]  # Desplazamiento hacia abajo
                )

            return (pl_left, pl_right, tuple(new_tiles))  # Nuevo estado con plungers actualizados

        # Unknown action
        return state  # Si la acción no es válida, devolvemos el mismo estado (sin cambios)

    def is_goal(self, state):  # Predicado de meta
        '''
        Define when a given state is a goal state
        '''
        return state == self.goal  # Compara el estado completo con la meta canónica

    def action_cost(self, s, a, s1):  # Coste uniforme por acción (para búsquedas)
        """
        Return the cost of a solution path that arrives at s1 from
        state s via action a.
        """
        return 1  # Uniform cost for all actions (todas las acciones cuestan 1)

### 2. Evalue su código con diferentes estrategias de búsqueda



Consulte el código en el repositorio de AIMA (https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) y utilice las implementaciones de búsqueda en amplitud y búsqueda en profundidad iterativa.

Pruebe varias configuraciones iniciales y pruebe cuál es la profundidad máxima que los métodos no informados pueden manejar.

In [ ]:
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)

    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)


class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost


failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.


def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)


def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None):
        return []
    return path_states(node.parent) + [node.state]

def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure

def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result


FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x):
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)

    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]

    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)

def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result


# Utility cost function for A*
def g(node):
    return node.path_cost

def best_first_search_with_metrics(problem, f):
    """Best-first search that also returns nodes expanded count."""
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    nodes_expanded = 0
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node, nodes_expanded
        for child in expand(problem, node):
            nodes_expanded += 1
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure, nodes_expanded

def depth_limited_search_with_metrics(problem, limit=10):
    """Depth-limited search that counts expansions."""
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    nodes_expanded = 0
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node, nodes_expanded
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                nodes_expanded += 1
                frontier.append(child)
    return result, nodes_expanded

def iterative_deepening_search_with_metrics(problem, max_limit=50):
    total_expanded = 0
    for limit in range(1, max_limit + 1):
        result, expanded = depth_limited_search_with_metrics(problem, limit)
        total_expanded += expanded
        if result != cutoff:
            return result, total_expanded, limit
    return cutoff, total_expanded, max_limit

# Heuristics
def heuristic_misplaced(state):
    """Cuenta fichas fuera de posiciones objetivo; 'L' indistinguibles.
    También añade coste si el estado de plungers no es (1,0).
    """
    pl_left, pl_right, tiles = state
    cost = (0 if pl_left == 1 else 1) + (0 if pl_right == 0 else 1)
    goal_positions = {
        'C': {7, 8, 14, 15},  # F,G,M,N indices
        'L': {2, 3, 4},       # A,B,C indices
        'B': {16},            # O
        'A': {17}             # P
    }
    for idx, t in enumerate(tiles):
        if t in goal_positions and idx not in goal_positions[t]:
            cost += 1
    return cost

def heuristic_manhattan(state):
    """Suma de distancias a cualquier posición válida del mismo tipo.
    Mapa 2D aproximado para la grilla dada.
    """
    _, _, tiles = state
    pos_coords = {
        0:(0,0), 1:(0,1),      # Q,R
        2:(1,0), 3:(1,1), 4:(1,2), 5:(1,3), 6:(1,4), 7:(1,5), 8:(1,6),
        9:(2,0),10:(2,1),11:(2,2),12:(2,3),13:(2,4),14:(2,5),15:(2,6),
        16:(3,1),17:(3,2)      # O,P roughly aligned
    }
    goal_positions = {
        'C': [7, 8, 14, 15],
        'L': [2, 3, 4],
        'B': [16],
        'A': [17]
    }
    total = 0
    for i, t in enumerate(tiles):
        if t in goal_positions:
            ci, cj = pos_coords[i]
            best = math.inf
            for gp in goal_positions[t]:
                gi, gj = pos_coords[gp]
                best = min(best, abs(ci-gi) + abs(cj-gj))
            total += 0 if best is math.inf else best
    return total

def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or (lambda node: 0)
    return best_first_search(problem, f=lambda n: g(n) + h(n))

def astar_search_with_metrics(problem, h):
    return best_first_search_with_metrics(problem, f=lambda n: g(n) + h(n))

def calculate_ebf(nodes_expanded, solution_depth):
    if solution_depth <= 0:
        return 0.0
    return (nodes_expanded + 1) ** (1.0 / solution_depth) - 1
# ==========================================
# 2) Evaluación con búsquedas no informadas
# ==========================================

def _apply_action(problem, state, action):
    return problem.result(state, action)

def _scramble_from_goal(k, rng):
    """Genera un estado alcanzable aplicando k movimientos aleatorios desde la meta."""
    probe = TripleCross_problem((1, 0, tuple(['E'] * 18)))
    state = probe.goal
    actions = ['R', 'L', '/']
    for _ in range(k):
        a = rng.choice(actions)
        state = _apply_action(probe, state, a)
    return state

def _run_ids_with_limit(problem, max_limit):
    """Versión acotada de IDS usando depth_limited_search."""
    for limit in range(1, max_limit + 1):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result
    return cutoff

def evaluate_uninformed_search(max_depth_to_test=7, ids_limit_per_case=12):
    """
    Prueba varias configuraciones (generadas desde la meta con k movimientos)
    y mide hasta qué profundidad resuelven BFS e IDS.
    """
    print("=" * 60)
    print("EVALUACIÓN: BÚSQUEDA EN AMPLITUD (BFS) E IDS")
    print("=" * 60)

    rng = random.Random(42)
    probe = TripleCross_problem((1, 0, tuple(['E'] * 18)))

    # Max target depth k for which a solution was found (regardless of len(solution))
    bfs_max_target_depth = 0
    ids_max_target_depth = 0

    for depth in range(1, max_depth_to_test + 1):
        init_state = _scramble_from_goal(depth, rng)
        problem = TripleCross_problem(init_state)

        print(f"\n--- Caso profundidad objetivo = {depth} ---")

        # BFS
        bfs_node = breadth_first_bfs(problem)
        if bfs_node is not failure:
            bfs_actions = path_actions(bfs_node)
            print(f"BFS: solución con {len(bfs_actions)} movimientos")
            bfs_max_target_depth = max(bfs_max_target_depth, depth)
        else:
            print("BFS: sin solución (failure)")

        # IDS (acotado)
        ids_node = _run_ids_with_limit(problem, max_limit=ids_limit_per_case)
        if ids_node is not failure and ids_node is not cutoff:
            ids_actions = path_actions(ids_node)
            print(f"IDS: solución con {len(ids_actions)} movimientos (límite {ids_limit_per_case})")
            ids_max_target_depth = max(ids_max_target_depth, depth)
        elif ids_node is cutoff:
            print(f"IDS: cutoff al límite {ids_limit_per_case}")
        else:
            print("IDS: sin solución (failure)")

    print("\n" + "-" * 60)
    print(f"Profundidad objetivo máxima con solución (BFS): {bfs_max_target_depth}")
    print(f"Profundidad objetivo máxima con solución (IDS): {ids_max_target_depth}")
    print(f"Profundidad objetivo máxima manejada por ambos: {min(bfs_max_target_depth, ids_max_target_depth)}")


if __name__ == '__main__':
    # Ejecutar evaluación del punto #2
    evaluate_uninformed_search()

    # Punto #3: Comparar A* (con heurísticas) vs IDS
    print("\n" + "=" * 60)
    print("EVALUACIÓN: A* con heurísticas vs IDS")
    print("=" * 60)
    rng = random.Random(7)
    heuristics = [("Misplaced", lambda n: heuristic_misplaced(n.state)),
                  ("Manhattan", lambda n: heuristic_manhattan(n.state))]

    for depth in range(1, 6):
        init = _scramble_from_goal(depth, rng)
        problem = TripleCross_problem(init)
        print(f"\n--- Caso (scramble k={depth}) ---")

        # IDS metrics
        ids_node, ids_expanded, ids_limit = iterative_deepening_search_with_metrics(problem, max_limit=20)
        if ids_node not in (failure, cutoff):
            ids_actions = path_actions(ids_node)
            ids_ebf = calculate_ebf(ids_expanded, len(ids_actions))
            print(f"IDS -> depth={len(ids_actions)}, expanded={ids_expanded}, ebf={ids_ebf:.3f}")
        else:
            print("IDS -> no solution or cutoff")

        # A* with heuristics
        for name, hfun in heuristics:
            node, expanded = astar_search_with_metrics(problem, hfun)
            if node is not failure:
                actions = path_actions(node)
                ebf = calculate_ebf(expanded, len(actions))
                print(f"A*[{name}] -> depth={len(actions)}, expanded={expanded}, ebf={ebf:.3f}")
            else:
                print(f"A*[{name}] -> failure")

    # ------------------------------
    # Test: Resolver ejemplo del profesor
    # ------------------------------
    def solve_professor_example():
        """Construye el estado observado en clase y lo resuelve con A*.

        Ajusta el diccionario `pos_to_tile` si tu foto/configuración difiere.
        Piezas: 'C' (círculo), 'L' (línea), 'B' (Binary), 'A' (Arts), 'E' vacío.
        Posiciones: 'Q','R','A'..'G','H'..'N','O','P'.
        """
        # Estimación razonable basada en la imagen proporcionada
        pos_to_tile = {
            # Líneas (tres barras amarillas en una fila central)
            'J': 'L', 'K': 'L', 'L': 'L',
            # Cuartos de círculo (marcados como 'C')
            'B': 'C', 'E': 'C', 'H': 'C', 'M': 'C',
            # Logos (ajustables si difieren)
            'O': 'B', 'P': 'A',
        }

        tmp = TripleCross_problem((1, 0, tuple(['E'] * 18)))
        tiles = ['E'] * 18
        for pos, t in pos_to_tile.items():
            tiles[tmp.positions[pos]] = t
        init_state = (1, 0, tuple(tiles))
        problem = TripleCross_problem(init_state)

        print("\n" + "=" * 60)
        print("RESOLUCIÓN DEL EJEMPLO DEL PROFESOR (A* Manhattan)")
        print("=" * 60)

        # Ejecutar A*
        node, expanded = astar_search_with_metrics(problem, lambda n: heuristic_manhattan(n.state))
        if node is failure:
            print("A*: no encontró solución; revisa el mapeo pos->pieza si la foto no coincide con el modelo.")
            return
        actions = path_actions(node)
        print(f"Acciones: {actions}")
        print(f"Longitud solución: {len(actions)} | Nodos expandidos: {expanded}")

        # Verificar que llega a la meta del modelo
        s = init_state
        for a in actions:
            s = problem.result(s, a)
        goal = TripleCross_problem((1, 0, tuple(['E'] * 18))).goal
        print(f"¿Alcanza meta canónica? {s == goal}")

    # Ejecutar el test del ejemplo
    solve_professor_example()

EVALUACIÓN: BÚSQUEDA EN AMPLITUD (BFS) E IDS

--- Caso profundidad objetivo = 1 ---
BFS: solución con 7 movimientos
IDS: solución con 7 movimientos (límite 12)

--- Caso profundidad objetivo = 2 ---
BFS: solución con 2 movimientos
IDS: solución con 2 movimientos (límite 12)

--- Caso profundidad objetivo = 3 ---
BFS: solución con 7 movimientos
IDS: solución con 7 movimientos (límite 12)

--- Caso profundidad objetivo = 4 ---
BFS: solución con 10 movimientos
IDS: solución con 10 movimientos (límite 12)

--- Caso profundidad objetivo = 5 ---
BFS: solución con 9 movimientos
IDS: solución con 9 movimientos (límite 12)

--- Caso profundidad objetivo = 6 ---
BFS: solución con 4 movimientos
IDS: solución con 4 movimientos (límite 12)

--- Caso profundidad objetivo = 7 ---
BFS: solución con 17 movimientos
IDS: cutoff al límite 12

------------------------------------------------------------
Profundidad objetivo máxima con solución (BFS): 7
Profundidad objetivo máxima con solución (IDS): 6
Prof

## 3. Implemente diferente heurísticas para el problema



Consulte el código en el repositorio de AIMA (https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) y utilice la implementación de A*.

Implemente al menos dos heurísticas admisibles y consistentes. Compare A * usando la heurística contra IDS calculando el número de nodos expandidos y el factor de ramificación efectivo, de la misma forma como se hace en la figura 3.29 de la 3ra edición de [Russell10].

In [ ]:
def nullHeuristic(state):
    return 0

def aStarSearch(problem, h=nullHeuristic):
    """
    Perform an A*t search on the problem.
    Returns the list of actions to reach the goal state.
    """
    ### your code here ###
    return []

def myHeuristic(state):
    ### your code here ###
    return 0